In [ ]:
%load_ext autoreload
%autoreload 2
#%matplotlib notebook

In [ ]:
import pdb
import math
from scipy import integrate
from scipy import special
import numpy as np
from cmath import *
import matplotlib.pyplot as plt
from IPython.display import display, Math, Latex
np.seterr(all='raise')
special.seterr(all='raise')
warpSize = 32
blockSize = 1024
from patch_geo_func import *

In [ ]:
den_area = 850 # mm^2 O'Kusky 1982 for the neuronal density we use later
# exp. measurement
darea = lambda E: 140*(0.78+E)**(-2.2) # Van Essen 1984 Vision Research "standard" map, 5232x1024
area_slice = lambda E: np.pi*E*darea(E)
# model fit
a = 0.635
b = 96.7
k = np.sqrt(140)*0.873145
s1 = 0.76
s2 = 0.1821
model_block = lambda p, e: model_block_ep(e,p,k,a,b,s1,s2)
half_stripe_width = 0 # to be read from elastic net simulation result

def model_slice(e):
    dslice = lambda p: model_block(p, e)
    r = integrate.quad(dslice, -np.pi/2, np.pi/2)
    return r[0]
ecc = np.hstack((np.linspace(0,2.5,50), np.linspace(2.5,90,100)))
ecc = ecc[1:]
fig = plt.figure('model_compare')
ax = fig.add_subplot(111)
model_s = np.array([model_slice(e) for e in ecc])
exp_s = np.array([area_slice(e) for e in ecc])
ax.plot(ecc, exp_s,'r')
ax.plot(ecc, model_s,'b')
ax2 = ax.twinx()
ax2.plot(ecc, np.abs(exp_s - model_s)/exp_s)

In [ ]:
# total v1 size
r = integrate.quad(area_slice,0,90)
v1_size_exp = r[0]
r = integrate.dblquad(model_block,0,90,-pi/2,pi/2)
v1_size_model = r[0]
display(Latex(rf'Size of V1 = {v1_size_exp:.3f} $mm^2$ (exp.) ~ {v1_size_model:.3f} $mm^2$ (model) ({v1_size_model*100/v1_size_exp:.3f}%)'))
ecc = 0.08 ########################################### <- set here 0.08 checked
r = integrate.quad(area_slice,0,ecc)
area_exp = r[0]
r = integrate.dblquad(model_block,0,ecc,-np.pi/2,np.pi/2)
area_model = r[0]
display(Latex(rf'A patch of v1 from eccentricity {0} to {ecc} degree yields {area_exp:.3f}  $mm^2$ (exp.) ~ {area_model:.3f}  $mm^2$ (model) ({area_model*100/area_exp:.3f}%)'))
model_ratio = den_area/v1_size_model
print(f'ratio: {model_ratio:.3f}')

In [ ]:
print('O''Kusky 1982; Kelly & Hawken 2017; M.Schmidt 2018, assuming constant density near the fovea')
print('##### L4 #####')
surface_den_L4Cbeta = 30000*model_ratio # per mm^2
display(Latex(rf'Rescale density 30000 per $mm^2$ in O\'Kusky 1982 to {surface_den_L4Cbeta:.3f} per $mm^2$ to keep constant number of neurons'))
assert(30000*den_area == surface_den_L4Cbeta*v1_size_model)
_n_L4Cbeta = area_model*surface_den_L4Cbeta # 32^4 * 10 ~ 10.4 million
n_L4Cbeta = int((_n_L4Cbeta+blockSize-1)//blockSize*blockSize)
nblock_L4Cbeta = n_L4Cbeta//blockSize
block_area_L4Cbeta = area_model/nblock_L4Cbeta
L4_area = area_model
print(f'Rounding number of neurons in L4Cbeta from {_n_L4Cbeta:.3f} to {n_L4Cbeta} = {nblock_L4Cbeta} x {blockSize}')
display(Latex(rf'area per block = {block_area_L4Cbeta:.3f} $mm^2$'))
display(Latex(rf'model density = {n_L4Cbeta/L4_area:.3f} per $mm^2$\n'))
print('##### L2/3 #####')
surface_den_L23 = 18000*model_ratio # per mm^2
display(Latex(rf'Rescale density 18000 per $mm^2$ in O\'Kusky 1982 to {surface_den_L23:.3f} per $mm^2$ to keep constant number of neurons'))
_n_L23 = area_model*surface_den_L23
n_L23 = int((_n_L23+blockSize-1)//blockSize*blockSize)
nblock_L23 = n_L23//blockSize
block_area_L23 = area_model/nblock_L23
L23_area = area_model
print(f'Rounding number of neurons in L2/3 from {_n_L23:.3f} to {n_L23} = {nblock_L23} x {blockSize}')
display(Latex(f'area per block = {block_area_L23:.3f} $mm^2$'))
display(Latex(f'model density = {n_L23/L23_area:.3f} per $mm^2$'))

In [ ]:
blockSize = 1024
fig, pos, xlim, ylim = plot_patch(a,b,k,ecc,half_stripe_width,nblock_L4Cbeta,ax=None,skip=602,s1=0.76,s2=0.1821,ret_fig=True,blockSize=blockSize)
#fig, pos = plot_patch(a,b,k,ecc,L4_area/32,32,ax=None,skip=602,s1=0.76,s2=0.1821,ret_fig=True,blockSize=1024)

In [ ]:
fig.savefig('fD_sp008.png', dpi = 600)

dim = 2
pos_filename = 'fD_sp008_2D.bin'
position_discrimination_digits = 1
with open(pos_filename, 'wb') as f:
    np.array([nblock_L4Cbeta, blockSize, dim]).astype('u4').tofile(f)        
    pos[:,:2,:].tofile(f) #f8
    
dim = 3
pos_filename = 'fD_sp008_3D.bin'
position_discrimination_digits = 1
with open(pos_filename, 'wb') as f:
    np.array([nblock_L4Cbeta, blockSize, dim]).astype('u4').tofile(f)        
    pos.tofile(f) #f8

In [ ]:
nblock = 32
blockSize = 1024
fig, pos, xlim, ylim = plot_patch(a,b,k,ecc,half_stripe_width,nblock,ax=None,skip=32,s1=0.76,s2=0.1821,ret_fig=True,blockSize=blockSize)

In [ ]:
fig.savefig('sobol_test_low_density.png', dpi = 600)
dim = 2
pos_filename = 'V1_pos_2D_lowDensity.bin'
with open(pos_filename, 'wb') as f:
    np.array([nblock, blockSize, dim]).astype('u4').tofile(f)        
    pos[:,:2,:].tofile(f) #f8
    
dim = 3
pos_filename = 'V1_pos_3D_lowDensity.bin'
with open(pos_filename, 'wb') as f:
    np.array([nblock, blockSize, dim]).astype('u4').tofile(f)
    pos.tofile(f)

In [ ]:
for i in range(pos.shape[0]):
    for j in range(pos.shape[2]):
        assert(np.sum((pos[i,:3,j].reshape(3,1) - pos[i,:3,:]==0).all(0)) == 1)

In [ ]:
%debug

characteristic length =

In [ ]:
cl = np.sqrt(block_area_L4Cbeta) * 1000
display(Latex(f'{cl:.3f} $\mu m$'))

In [ ]:
md = np.power(cl*cl*100/1024,1/3)
display(Latex(f'~204x204x100 $\mu m^3$ (1024 neurons) per block, average inter-neuron(soma)-distance = {md:.3f} $\mu m$'))

conduction velocity ~ 1m/s, being unmylineated horizontal connections  
(0.3m/s for L2/3 and upper L4) <i>Girard et al J Neurophysiol 2001</i>  
minmum delay = 0.016 ms, no spike-correction is needed for time step (dt) < minimum delay
100Hz instantaneous firing rate, 1000 connections leads to ~ 1 spike every 0.01ms. 

In [ ]:
dt = 1 #ms
nob = 1000/cl
print(nob)

dt = 1ms means neurons in the nearest 5~6 blocks needs to be considered for spike correction  
dt = 0.125ms (1/8) suffice for spike correction within block, #ASSUMPTION# nearfield spike correction only (neurons at block boundarys are neglected) spikes send to the other blocks are resolved after the current step.

#dt = 0.125ms is thus limited by the number of threads per block in NVIDIA GPU  
To further increase dt, will need a cross block spike-correction with CPU, or wait for an update from NVIDIA, otherwise results in an extra increase in spike-correction error, maybe tolerable.

In [ ]:
time = 500
dt = 0.0625
batch_time_per_dt = 500e-6 #sec
nstep = time//dt
blocks = 10240
resident_blocks = 8
time_cost = np.int(blocks/resident_blocks) * batch_time_per_dt * nstep
print('simulate for', time ,'ms with dt =', dt, 'ms cost', time_cost/3600,'h')

In [ ]:
pos = np.fromfile('server_data/test_3d_pos.bin')

In [ ]:
pos = np.reshape(pos,(3,nblock_L4Cbeta,blockSize))

In [ ]:
from scipy import integrate
# Maplpeli et al 1996
parvo_den_alt = lambda E: 1011688*(E+2.9144)**(-2.6798) # cells/deg^2
# offset (transition between crossed and uncrossed LGN Chalupa and Lia 1991, Lavidor and Walsh 2004)
#offset = 0.5/
#parvo_slice = lambda E: (np.pi+offset)*E*parvo_den_alt(E)
parvo_slice = lambda E: np.pi*E*parvo_den_alt(E)
ecc_Connelly = 2.5
r = integrate.quad(parvo_slice, 0, ecc_Connelly)
nparvo_connelly = round(r[0])
print(f'{nparvo_connelly} parvo cells inside {ecc_Connelly} in total > 110,000 connelly and van essen 1984')
magno_den_alt = lambda E: 2620.2*((E-1.8322)**2+5.5638)**(-0.8012) # cells/deg^2
# in the cortex sheet
# R dendritic spread of L4 neuron
# r dendritic spread of LGN neuron
R_beta = 0.05 # mm Lund 1973
R_alpha = 0.1 # mm
r_parvo = 0.1 # mm Blasdel and lund 1983
r_magno = 0.25 # mm

def M_V1(e):
    dslice = lambda p: model_block(p, e)
    meanM = integrate.quad(dslice, -np.pi/2, np.pi/2)
    return meanM[0]/np.pi

# lgn per cortex cell = den_alt(ecc) * (R+r)^2 * darea(ecc)
#parvo_beta = lambda e: parvo_den_alt(e) * np.power((R_beta + r_parvo),2) * M_V1(e)
spread = np.power((R_beta + r_parvo),2)
parvo_beta = lambda e: parvo_den_alt(e) * spread / M_V1(e)
#parvo_beta = lambda e: 24610 * M_V1(e) / parvo_den_alt(e)

x_ecc = x_ep(ecc,0,k,a,b) + r_parvo
ecc0 = e_x(x_ecc,k,a,b)
print(f'V1 occupies {ecc} eccentricity, parvo LGN occupies {ecc0} eccentricity')
#ratio_L56 = 0.7 #L5,6 only is not correct, use 1.0, sked Bob
ratio_CI = 0.66 # C:I ~ 4:1 (Connolly and Van Essen 1984) within 2.5 degree
r = integrate.quad(parvo_slice, 0, ecc0)
nparvo0_beta = round(r[0])
ecc = np.linspace(0,ecc0,101)
ecc = ecc[1:]
fig = plt.figure('parvo per 4Cbeta',  dpi = 600)
ax = fig.add_subplot(121)
ax.set_title(f'parvo per 4Cbeta, spread = {spread:.4f}')
ax.plot(ecc, [parvo_beta(e)*ratio_CI for e in ecc], ':b')
ax.plot(ecc, [parvo_beta(e)*(1-ratio_CI) for e in ecc], ':r')
#ax.set_yscale('log')
ax2 = ax.twinx()
#ax2.plot(ecc, [M_V1(e) for e in ecc], 'r')
ax2.plot(ecc, [parvo_den_alt(e)*ratio_CI/1000 for e in ecc], ':g')
ax2.plot(ecc, [parvo_den_alt(e)*(1-ratio_CI)/1000 for e in ecc], ':m')
ax2.plot(ecc, [M_V1(e) for e in ecc], 'k')

def parvo_nLGN_beta(ecc, ratio, ecc0):
    # dE determined in a iteratively forward way.
    count = 1
    if ecc > 0:
        decc = 0
        new_ecc = ecc
        delta = 1
        tol = 1e-6
        while delta>tol and count < 100:
            new_decc = 1/np.sqrt(parvo_den_alt(new_ecc)*ratio)
            new_ecc = ecc + new_decc/2
            delta = abs(decc - new_decc)
            decc = new_decc
            if ecc + decc > ecc0:
                #decc = ecc0 - ecc
                break
            count = count + 1
        assert(count < 50)
    else:
        decc = np.sqrt(2/(np.pi*parvo_den_alt(0)*ratio))
    r = integrate.quad(parvo_slice, ecc, ecc+decc)
    #print(decc, r[0], r[0]*ratio)
    return np.round(r[0]*ratio).astype(int), ecc + decc

def spread_VF(ecc0, ratio, n0):
    current_ecc = 0
    sliced_N = np.array([], dtype = int)
    sliced_ecc = np.array([])
    N = 0
    decc = np.array([])
    while current_ecc < ecc0:
        old_ecc = current_ecc
        sN, current_ecc = parvo_nLGN_beta(current_ecc, ratio, ecc0)
        sliced_ecc = np.append(sliced_ecc, (old_ecc + current_ecc)/2)
        decc = np.append(decc, current_ecc - old_ecc)
        N = N + sN
        sliced_N = np.append(sliced_N, sN)
    return N, sliced_N, sliced_ecc, decc, current_ecc

nparvo_I5, Isliced_N, Isliced_ecc, Idecc, Icurrent_ecc = spread_VF(ecc0, (1-ratio_CI), nparvo0_beta)
nparvo_C6, Csliced_N, Csliced_ecc, Cdecc, Ccurrent_ecc = spread_VF(ecc0, ratio_CI, nparvo0_beta)
print(f'Cecc = {Ccurrent_ecc}, Iecc = {Icurrent_ecc}')
ecc0 = max(Icurrent_ecc, Ccurrent_ecc)

ax = fig.add_subplot(122)
ax2 = ax.twinx()
ax.plot(Isliced_ecc, Isliced_N, ':r')
ax.plot(Csliced_ecc, Csliced_N, ':b')
ax2.plot(Isliced_ecc, Idecc*60, ':m')
ax2.plot(Csliced_ecc, Cdecc*60, ':g')

ax2.set_ylabel('VF radius in min')
nparvo_beta = nparvo_I5 + nparvo_C6
print(f'reached {Icurrent_ecc}, {Ccurrent_ecc} with {nparvo_beta} ~ {round(nparvo0_beta)} at {ecc0}')

In [ ]:
def assign_pos_slice(n, ecc):
    equi_theta = np.linspace(-np.pi/2, np.pi/2, n+1)
    theta = (equi_theta[1:] + equi_theta[:-1])/2
    cartPos = np.zeros((2,n))
    polarPos = np.zeros((2,n))
    polarPos[0,:] = theta
    polarPos[1,:] = ecc
    cartPos[0,:] = np.cos(theta)*ecc
    cartPos[1,:] = np.sin(theta)*ecc
    return polarPos, cartPos

def assign_pos(sliced_N, sliced_ecc, n):
    polarPos = np.zeros((2,n))
    cartPos = np.zeros((2,n))
    nslice = sliced_N.size
    current_n = 0
    for i in range(nslice):
        polarPos[:,current_n:current_n + sliced_N[i]], cartPos[:,current_n:current_n + sliced_N[i]] = assign_pos_slice(sliced_N[i], sliced_ecc[i])
        current_n = current_n + sliced_N[i]
    return polarPos, cartPos

def spread_ecc(n, ecc):
    ecc0 = np.hstack((0,ecc))
    old_area = np.pi*(ecc0[1:]*ecc0[1:] - ecc0[:-1]*ecc0[:-1])
    density = n/old_area
    mean_density = np.mean(density)
    new_area = n/mean_density
    area0 = np.sum(old_area)
    tol = 1e-10
    darea = np.sum(new_area) - area0
    old_density = mean_density
    count = 0
    print(f'start out with {darea}, {mean_density}')
    while np.abs(darea) > 1e-10:
        if darea > 0:
            new_density = old_density * (1 - 0.5*darea/area0)
        else:
            new_density = old_density * (1 + 0.5*darea/area0)
        old_density = new_density
        new_area = n/new_density
        darea = np.sum(new_area) - area0
        count = count + 1
        if count > 100:
            print(f'100 loops out with {new_density}, {darea}')
        
    uniform_ecc = np.zeros(n.size, dtype = float)
    uniform_ecc[0] = np.sqrt(new_area[0]/np.pi)
    for i in range(n.size-1):
        uniform_ecc[i+1] = np.sqrt(new_area[i+1]/np.pi + uniform_ecc[i]*uniform_ecc[i])
    print(f'new ecc {uniform_ecc[-1]}, old ecc {ecc[-1]}, after {count} adjustments')
    return uniform_ecc

def assign_pos_uniform(sliced_N, sliced_ecc, n):
    sliced_ecc_uniform = spread_ecc(sliced_N, sliced_ecc)
    #print(sliced_ecc_uniform)
    #_sliced_ecc_uniform = np.linspace(sliced_ecc[0], sliced_ecc[-1], sliced_ecc.size)
    #print(_sliced_ecc_uniform)
    cartPos = np.zeros((2,n))
    nslice = sliced_N.size
    current_n = 0
    for i in range(nslice):
        _, cartPos[:,current_n:current_n + sliced_N[i]] = assign_pos_slice(sliced_N[i], sliced_ecc_uniform[i])
        current_n = current_n + sliced_N[i]
        
    assert(np.max(np.sqrt(cartPos[0,:]*cartPos[0,:] + cartPos[1,:]*cartPos[1,:])) <= sliced_ecc[-1])
    return cartPos

In [ ]:
# Excluding S_on, S_off from LGN Koniocellular layers to L1, L2/3 for now
# C-On:Off ~ 3:1; I-On:Off ~ 2:1 (Wiesel and Hubel 1966, Schiller-and-Malpeli-1978, Derrington and Lennie 1984, Martin-and-Lee 2014), cone-to-ganglion ~ 1:1 (McMahon et al., 2000)
# Red:Green, 1.0 central (Bowmaker 2003, cite-only.bib) 1.6 overall(Deeb et al., 2000, cite-only.bib)
rL_on = 0
rL_off = 1
rM_on = 2
rM_off = 3
np.random.seed(74381023)
#                       L                         M
#C_sequence = np.array([rL_on,rL_on,rL_off,rL_on, rM_on,rM_on,rM_off,rM_on])
#I_sequence = np.array([rL_on,rL_on,rL_off,       rM_on,rM_on,rM_off])
C_sequence = np.array([rL_on,rL_on,rM_off,rL_on,rM_on,rM_on,rL_off,rM_on])
I_sequence = np.array([rL_on,rM_off,rL_on,rM_on,rL_off,rM_on])

parvo_pos_I5, parvo_pos_I5_cart= assign_pos(Isliced_N, Isliced_ecc, nparvo_I5)
parvo_pos_C6, parvo_pos_C6_cart= assign_pos(Csliced_N, Csliced_ecc, nparvo_C6)

parvo_pos_I5_uniform= assign_pos_uniform(Isliced_N, Isliced_ecc, nparvo_I5)
parvo_pos_C6_uniform= assign_pos_uniform(Csliced_N, Csliced_ecc, nparvo_C6)

def shuffle_fill(seq, arr):
    seqSize = seq.size
    n = arr.size
    for i in range(n//seqSize+1):
        current_i = i*seqSize
        next_i = current_i + seqSize
        if next_i > n:
            arr[current_i:n] = np.random.permutation(seq)[:n-current_i]
        else:
            arr[current_i:next_i] = np.random.permutation(seq)

RG_OnOff_I5 = np.zeros(nparvo_I5)
shuffle_fill(C_sequence, RG_OnOff_I5)

RG_OnOff_C6 = np.zeros(nparvo_C6)
shuffle_fill(I_sequence, RG_OnOff_C6)

In [ ]:
# reorder by coneType
def reorder(arr, ref, n):
    new_arr = np.zeros(arr.shape)
    offset = 0
    for i in range(n):
        pick = ref == i
        npick = np.sum(pick)
        new_arr[:,offset:(offset+npick)] = arr[:,pick].copy()
        offset += npick
    assert(offset == arr.shape[1])
    return new_arr

def ref_reorder(ref, n):
    new_ref = np.zeros(ref.shape)
    offset = 0
    for i in range(n):
        pick = ref == i
        npick = np.sum(pick)
        new_ref[offset:(offset+npick)] = ref[pick].copy()
        offset += npick
    assert(offset == ref.size)
    return new_ref

parvo_pos_I5 = reorder(parvo_pos_I5, RG_OnOff_I5, 4)
parvo_pos_I5_cart = reorder(parvo_pos_I5_cart, RG_OnOff_I5, 4)
parvo_pos_I5_uniform = reorder(parvo_pos_I5_uniform, RG_OnOff_I5, 4)
RG_OnOff_I5 = ref_reorder(RG_OnOff_I5, 4)
parvo_pos_C6 = reorder(parvo_pos_C6, RG_OnOff_C6, 4)
parvo_pos_C6_cart = reorder(parvo_pos_C6_cart, RG_OnOff_C6, 4)
parvo_pos_C6_uniform = reorder(parvo_pos_C6_uniform, RG_OnOff_C6, 4)
RG_OnOff_C6 = ref_reorder(RG_OnOff_C6, 4)
ntypeC6 = np.zeros(4, dtype = int)
ntypeI5 = np.zeros(4, dtype = int)
offset5 = 0
offset6 = 0
for i in range(4):
    ntypeI5[i] = np.sum(RG_OnOff_I5==i)
    assert((RG_OnOff_I5[offset5:(offset5+ntypeI5[i])]-i == 0).all())
    offset5 = offset5 + ntypeI5[i]
    ntypeC6[i] = np.sum(RG_OnOff_C6==i)
    assert((RG_OnOff_C6[offset6:(offset6+ntypeC6[i])]-i == 0).all())
    offset6 = offset6 + ntypeC6[i] 

fig = plt.figure('LGN_pos', dpi = 300)
ax = fig.add_subplot(121)
offset = 0
ax.plot(parvo_pos_C6_cart[0,offset:ntypeC6[0]], parvo_pos_C6_cart[1,offset:ntypeC6[0]], ',r', ms = 0.01)
offset = offset + ntypeC6[0]
ax.plot(parvo_pos_C6_cart[0,offset:(offset+ntypeC6[1])], parvo_pos_C6_cart[1,offset:(offset+ntypeC6[1])], ',m', ms = 0.01)
offset = offset + ntypeC6[1]
ax.plot(parvo_pos_C6_cart[0,offset:(offset+ntypeC6[2])], parvo_pos_C6_cart[1,offset:(offset+ntypeC6[2])], ',g', ms = 0.01)
offset = offset + ntypeC6[2]
ax.plot(parvo_pos_C6_cart[0,offset:(offset+ntypeC6[3])], parvo_pos_C6_cart[1,offset:(offset+ntypeC6[3])], ',y', ms = 0.01)
ax.set_aspect('equal')
ax2 = fig.add_subplot(122)
ax2.plot(parvo_pos_C6_cart[0,RG_OnOff_C6 == 0], parvo_pos_C6_cart[1,RG_OnOff_C6 == 0], ',r', ms = 0.01)
ax2.plot(parvo_pos_C6_cart[0,RG_OnOff_C6 == 1], parvo_pos_C6_cart[1,RG_OnOff_C6 == 1], ',m', ms = 0.01)
ax2.plot(parvo_pos_C6_cart[0,RG_OnOff_C6 == 2], parvo_pos_C6_cart[1,RG_OnOff_C6 == 2], ',g', ms = 0.01)
ax2.plot(parvo_pos_C6_cart[0,RG_OnOff_C6 == 3], parvo_pos_C6_cart[1,RG_OnOff_C6 == 3], ',y', ms = 0.01)
#ax2.plot(parvo_pos_C6_uniform[0,RG_OnOff_C6 == 0], parvo_pos_C6_uniform[1,RG_OnOff_C6 == 0], ',r', ms = 0.01)
#ax2.plot(parvo_pos_C6_uniform[0,RG_OnOff_C6 == 1], parvo_pos_C6_uniform[1,RG_OnOff_C6 == 1], ',m', ms = 0.01)
#ax2.plot(parvo_pos_C6_uniform[0,RG_OnOff_C6 == 2], parvo_pos_C6_uniform[1,RG_OnOff_C6 == 2], ',g', ms = 0.01)
#ax2.plot(parvo_pos_C6_uniform[0,RG_OnOff_C6 == 3], parvo_pos_C6_uniform[1,RG_OnOff_C6 == 3], ',y', ms = 0.01)
#ax2.plot(parvo_pos_I5_cart[0,RG_OnOff_I5 == 0], parvo_pos_I5_cart[1,RG_OnOff_I5 == 0], ',r')
#ax2.plot(parvo_pos_I5_cart[0,RG_OnOff_I5 == 1], parvo_pos_I5_cart[1,RG_OnOff_I5 == 1], ',m')
#ax2.plot(parvo_pos_I5_cart[0,RG_OnOff_I5 == 2], parvo_pos_I5_cart[1,RG_OnOff_I5 == 2], ',g')
#ax2.plot(parvo_pos_I5_cart[0,RG_OnOff_I5 == 3], parvo_pos_I5_cart[1,RG_OnOff_I5 == 3], ',y')
ax2.set_aspect('equal')

In [ ]:
ecc = np.log(np.linspace(1,np.exp(3.0),10))
k = 0.2049795945022049 
log_cpd0 = 3.6741080244555278
cpd = np.exp(-k*ecc + log_cpd0)
fig = plt.figure()
ax = fig.add_subplot(111)
ax.plot(ecc, cpd)

In [ ]:
def acuity(ecc):
    k = 0.2049795945022049 
    log_cpd0 = 3.6741080244555278
    cpd = np.exp(-k*ecc + log_cpd0)
    return 1/cpd/4

def polar2cart(r, n = 20):
    theta = np.linspace(0, 2*np.pi, n)
    return r*np.cos(theta), r*np.sin(theta)
sampleSize = 10
iSample = np.random.randint(nparvo_C6, size = sampleSize)
rSample = acuity(parvo_pos_C6[1, iSample])

fig = plt.figure('LGN_pos', dpi = 300)
ax = fig.add_subplot(121)
ax.plot(parvo_pos_C6_cart[0,RG_OnOff_C6 == 0], parvo_pos_C6_cart[1,RG_OnOff_C6 == 0], ',r', ms = 0.01)
ax.plot(parvo_pos_C6_cart[0,RG_OnOff_C6 == 1], parvo_pos_C6_cart[1,RG_OnOff_C6 == 1], ',m', ms = 0.01)
ax.plot(parvo_pos_C6_cart[0,RG_OnOff_C6 == 2], parvo_pos_C6_cart[1,RG_OnOff_C6 == 2], ',g', ms = 0.01)
ax.plot(parvo_pos_C6_cart[0,RG_OnOff_C6 == 3], parvo_pos_C6_cart[1,RG_OnOff_C6 == 3], ',y', ms = 0.01)
for i in range(sampleSize):
    x, y = polar2cart(rSample[i])
    ax.plot(parvo_pos_C6_cart[0,iSample[i]] + x, parvo_pos_C6_cart[1,iSample[i]] + y, '-k', lw = 0.01)
    
ax.set_aspect('equal')
ax2 = fig.add_subplot(122)
ax2.plot(parvo_pos_C6_uniform[0,RG_OnOff_C6 == 0], parvo_pos_C6_uniform[1,RG_OnOff_C6 == 0], ',r', ms = 0.01)
ax2.plot(parvo_pos_C6_uniform[0,RG_OnOff_C6 == 1], parvo_pos_C6_uniform[1,RG_OnOff_C6 == 1], ',m', ms = 0.01)
ax2.plot(parvo_pos_C6_uniform[0,RG_OnOff_C6 == 2], parvo_pos_C6_uniform[1,RG_OnOff_C6 == 2], ',g', ms = 0.01)
ax2.plot(parvo_pos_C6_uniform[0,RG_OnOff_C6 == 3], parvo_pos_C6_uniform[1,RG_OnOff_C6 == 3], ',y', ms = 0.01)
ax2.set_aspect('equal')

In [ ]:
prec = 'float32'
#for assign_attr.py
theme = 'micro2'
parvo_pos_I5_file = 'parvo_pos_I5-'+theme+'.bin'
with open(parvo_pos_I5_file, 'wb') as f:
    np.array([nparvo_I5], dtype = 'u4').tofile(f)
    np.array([ecc0], dtype = 'f8').tofile(f)
    parvo_pos_I5_uniform.tofile(f)
    RG_OnOff_I5.astype('u4').tofile(f)

parvo_pos_C6_file = 'parvo_pos_C6-'+theme+'.bin'
with open(parvo_pos_C6_file, 'wb') as f:
    np.array([nparvo_C6], dtype = 'u4').tofile(f)
    np.array([ecc0], dtype = 'f8').tofile(f)
    parvo_pos_C6_uniform.tofile(f)
    RG_OnOff_C6.astype('u4').tofile(f)

In [ ]:
"""
parvo_pos_I5_file = 'parvo_pos_I5.bin'
with open(parvo_pos_I5_file, 'wb') as f:
    np.array([nparvo_I5], dtype = 'u4').tofile(f)
    RG_OnOff_I5.astype('int32').tofile(f)
    parvo_pos_I5.astype(prec).tofile(f)
    
parvo_pos_C6_file = 'parvo_pos_C6.bin'
with open(parvo_pos_C6_file, 'wb') as f:
    np.array([nparvo_C6], dtype = 'u4').tofile(f)
    RG_OnOff_C6.astype('int32').tofile(f)
    parvo_pos_C6.astype(prec).tofile(f)
""" 
prec = 'float32'
LR_merged_file = 'parvo_float-'+theme+'.bin'
with open(LR_merged_file, 'wb') as f:
    # Ipsi first
    np.array([nparvo_I5], dtype = 'u4').tofile(f)
    np.array([nparvo_C6], dtype = 'u4').tofile(f)
    np.array([ecc0], dtype = prec).tofile(f)
    # cart
    np.array([0, ecc0, -ecc0, 2*ecc0], dtype = prec).tofile(f)
    merged = np.hstack((parvo_pos_I5_cart, parvo_pos_C6_cart))
    assert(merged.shape[0] == 2)
    assert(merged.shape[1] == nparvo_I5 + nparvo_C6)
    merged.astype(prec).tofile(f)
    # type
    np.hstack((RG_OnOff_I5, RG_OnOff_C6)).astype('u4').tofile(f)
    # polar
    merged = np.hstack((parvo_pos_I5, parvo_pos_C6))
    assert(merged.shape[0] == 2)
    assert(merged.shape[1] == nparvo_I5 + nparvo_C6)
    merged.astype(prec).tofile(f)
    
print(f'#ipsi = {nparvo_I5}, #contra = {nparvo_C6}')
print(f"{min(parvo_pos_I5[1,:])}, {min(parvo_pos_C6[1,:])}")
print(f"{max(parvo_pos_I5[1,:])}, {max(parvo_pos_C6[1,:])}")

In [ ]:
# reorder old LGN_surfaceID.bin, one-time usage
nLGN = nparvo_I5 + nparvo_C6
with open('LGN_surfaceID.bin', 'rb') as f:
    max_id = np.fromfile(f, 'u4', 2)
    LGN_surfaceID = np.fromfile(f, 'i4', 2*nLGN).reshape(2,nLGN)
print(max_id)
LGN_id_I5 = LGN_surfaceID[:, :nparvo_I5]
LGN_id_C6 = LGN_surfaceID[:, nparvo_I5:]
LGN_newID = np.zeros((2,nLGN), dtype = 'i4')
LGN_newID[:, :nparvo_I5] = reorder(LGN_id_I5, RG_OnOff_I5, 4)
LGN_newID[:, nparvo_I5:] = reorder(LGN_id_C6, RG_OnOff_C6, 4)

fig = plt.figure('LGN_ID', dpi = 1000)
ax = fig.add_subplot(111)
ax.plot(LGN_newID[0,:],LGN_newID[1,:],',r', ms = 0.1)
ax.set_aspect('equal')
with open('LGN_surfaceID.bin', 'wb') as f:
    max_id.astype('u4').tofile(f)
    LGN_newID.tofile(f)

In [ ]:
def area(raxn, rden, d):
    area = np.zeros(d.size)
    minr = max(raxn,rden) - min(raxn,rden)
    maxr = max(raxn,rden) + min(raxn,rden)
    area[d <= minr] = np.power(min(raxn,rden),2)*np.pi
    mid_pick = np.logical_and(minr < d, d < maxr)
    d_mid = d[mid_pick]
    cos_theta_axn = (raxn*raxn + d_mid*d_mid - rden*rden)/(2*raxn*d_mid)
    cos_theta_den = (rden*rden + d_mid*d_mid - raxn*raxn)/(2*rden*d_mid)
    seg_axn = np.arccos(cos_theta_axn)*raxn*raxn
    seg_den = np.arccos(cos_theta_den)*rden*rden

    chord_axn = np.sqrt(raxn*raxn - np.power(cos_theta_axn*raxn,2)) * raxn * cos_theta_axn
    chord_den = np.sqrt(rden*rden - np.power(cos_theta_den*rden,2)) * rden * cos_theta_den
    area[mid_pick] = seg_axn+seg_den-chord_axn-chord_den
    assert(np.sum(area<0) == 0)
    return area

def base(raxn, rden, d):
    #subr = d-raxn
    #subr[subr<0] = 0
    #supr = d+raxn
    #supr[supr>rden] = rden
    #base = np.pi*(np.power(supr,2) - np.power(subr,2))
    base = np.pi*(np.power(raxn,2))
    return base

In [ ]:
raxn = 100 #um
rden = 100
nd = 100
dd = (raxn+rden)/nd
dee = np.linspace(0,raxn+rden-dd,nd-1)

fig = plt.figure('prob')
ax = fig.add_subplot(111)
ratio = 0.095
ax.plot(dee,  area(raxn,rden,dee)/base(raxn,rden,dee)*ratio)
x = np.array([12.5, 37.5, 75, 150])
y = np.array([8.8, 7.0, 5.0, 1.3])/100
ax.plot(x,y)
ax.set_xlim(x[0],x[-1])

In [ ]:
raxn = 100
rden = 80
nd = 100
dd = (raxn+rden)/nd
dii = np.linspace(0, raxn+rden-dd,nd-1)
fig = plt.figure('prob')
ax = fig.add_subplot(211)
y = area(raxn,rden,dii)/base(raxn,rden,dii)
baseline = -0.1
ratio = 1
ax.plot(dii,y*ratio+baseline)
x = [12.5, 37.5, 75]
y = [0.72, 0.50, 0.55]
yr = [0.51, 0.33, 0.27]
ax.plot(x,y)
ax.plot(x,yr)
ax.set_xlim(x[0],x[-1])

In [ ]:
#Connelly-Van_Essen-1984 
Ap = 8.37
Bp = 1.28
Cp = -1.96
parvo_den = lambda E: Ap*(E+Bp)**Cp*10000 # cells/deg^2
Am = 3.52
Bm = 3.1
Cm = -1.56
magno_den = lambda E: Am*(E+Bm)**Cm*1000# cells/deg^2
deg = 5
#consistent Maplpeli et al 1996
parvo_den_alt = lambda E: 1011688*(E+2.9144)**(-2.6798)
magno_den_alt = lambda E: 2620.2*((E-1.8322)**2+5.5638)**(-0.8012)
fig = plt.figure('lgn cell areal mf', dpi=150)
ax = fig.add_subplot(221)
ecc = np.arange(0,90)
ax.plot(ecc, magno_den(ecc))
ax.plot(ecc, magno_den_alt(ecc),':')
ax.set_title(f'{magno_den(deg):.1f}, {magno_den_alt(deg):.1f}')
ax.set_ylabel('magno den')
ax.set_yscale('log')
ax = fig.add_subplot(222)
ax.plot(ecc, parvo_den(ecc))
ax.plot(ecc, parvo_den_alt(ecc),':')
ax.set_yscale('log')
ax.set_title(f'{parvo_den(deg):.1f}, {parvo_den_alt(deg):.1f}')
ax.set_ylabel('parvo den') 
est_4Cbeta_over_4A = 0.6
#blasdel and lund 1983,
magno_spread = 0.5 # mm (mean) 
parvo_spread = 0.1 # mm (upper limit)
surface_den_L4Calpha = 17300
magno_spread_reach_max = surface_den_L4Calpha*magno_spread**2
parvo_spread_reach_max = surface_den_L4Cbeta*parvo_spread**2
ax = fig.add_subplot(223)
ax.plot(ecc, magno_den(ecc)/darea(ecc,0)*magno_spread**2)
ax.plot(ecc, magno_den_alt(ecc)/darea(ecc,0)*magno_spread**2, ':')
ax.set_ylabel('magno per 4Calpha')
ax.set_title(f'{magno_den(deg)/darea(deg,0)*magno_spread**2:.1f}, {magno_den_alt(deg)/darea(deg,0)*magno_spread**2:.1f}')
ax = fig.add_subplot(224)
ax.plot(ecc, parvo_den(ecc)/darea(ecc,0)*parvo_spread**2)
ax.plot(ecc, parvo_den_alt(ecc)/darea(ecc,0)*parvo_spread**2, ':')
ax.set_ylabel('parvo per 4A and/or 4Cbeta')
ax.set_title(f'{parvo_den(deg)/darea(deg,0)*parvo_spread**2:.1f}, {parvo_den_alt(deg)/darea(deg,0)*parvo_spread**2:.1f}')
plt.tight_layout()

In [ ]:
fig.savefig('upper_limit_of_nLGN_per_V1.png')

In [ ]:
deg = 5
p_den = lambda E: 110000*(E+1.28)**(-1.96) #cell/deg^2
m_den = lambda E: 4600*(E+3.12)**(-1.56) #cell/deg^2
print(p_den(deg),m_den(deg))
areal_cmf = lambda E: 12.2**2*(E+0.94)**(-2.0) # mm^2/deg^2
print(areal_cmf(deg))
parvo_per_p = p_den(deg)/areal_cmf(deg)
print(f'{parvo_per_p:.1f} P cells/mm^2 to 4Cbeta + 4A')
magno_per_m = m_den(deg)/areal_cmf(deg)
print(f'{magno_per_m:.1f} M cells/mm^2 to 4Calpha')

In [ ]:
fig = plt.figure('Schein')
ax = fig.add_subplot(111)
ax.plot(ecc, p_den(ecc)/areal_cmf(ecc))
ax.plot(ecc, m_den(ecc)/areal_cmf(ecc),':')